# Calculate phase mismatch

(For a slightly roundabout way) To calculate the phase mismatch I can use the `lalapps_pulsar_frequency_evolution` code. If, for example, I create a fake pulsar close to the ecliptic (in this is roughly in [Pisces](https://en.wikipedia.org/wiki/Pisces_(constellation))) with a par file that looks like:

```
PSRJ test
F0 500
RAJ 01:00:00.00
DECJ 15:00:00
PEPOCH 50000
UNITS TDB
EPHEM DE405

```
so that the emission frequency is at 1000 Hz, I can run:

```
lalapps_pulsar_frequency_evolution --detector H1 --par-file /home/matthew/tmp/pisces.par --start 900000000 --deltat 60 --timespan 3153600 --output-dir .
```

to create a `frequency_evolution_H1.txt` file containing the frequency evolution over a year in one minute steps.

I can then edit the par file to give a slightly different position, e.g., with a change such that `RAJ 01:00:00.02`, and re-run `lalapps_pulsar_frequency_evolution`. If I'd copied the original output file to, e.g., `frequency_evolution_H1_1.txt`, then I can calculate the phase mismatch over the year for the two positions in python via:

```python
import numpy as np

# read in the frequency evolutions for the two different positions
f1 = np.loadtxt('frequency_evolution_H1_1.txt')
f2 = np.loadtxt('frequency_evolution_H1.txt')

# create arrays to hold to phase evolution
phases1 = np.zeros(len(f1))
phases2 = np.zeros(len(phases1))

# calculate the phase evolution
for i in range(1, len(phases1)):
    phases1[i] = phases1[i-1] + 2.*np.pi*f1[i-1,1]*(f1[i,0]-f1[i-1,0])
    
for i in range(1, len(phases1)):
    phases2[i] = phases2[i-1] + 2.*np.pi*f2[i-1,1]*(f2[i,0]-f2[i-1,0])
    
# calculate the amplitude mismatch caused by the different positions
T = f1[-1,0]-f1[0,0] # observation time
dt = f1[1,0]-f1[0,0] # time steps
mm = (1. - np.trapz(np.cos(phases2-phases1), dx=dt)/T)
```

For the above cases, i.e. a signal at 1000 Hz, a roughly 10% mismatch is caused by an $1.5 \mu$rad sky position difference. A ten times larger sky position difference is allowed for signals at 100 Hz.